### Importing all relevant libraries

In [ ]:
%matplotlib inline
from Functions import A1, A2
import numpy as np
import pandas as pd
import sobol_seq
import matplotlib.pyplot as plt
from pandas import ExcelWriter
plt.style.use('ggplot')

### Defining the functions

In [ ]:
k = 6

a2 = np.array([0,0.5,3,9,99,99])

run = 50

p = 13

functions = [A1, A2]

AE_df = pd.read_excel('AE_df.xlsx',index_col=0) #the analytical values

df3 = pd.read_csv('2019.03.05_scrambled.csv').T.iloc[1:].T

df4 = pd.DataFrame(sobol_seq.i4_sobol_generate(2*k, run*(2**p+5*2**(p-4))))

df = pd.read_csv('2019.04.24.index.csv',index_col=0)

df5 = df4.reindex(df.index).reset_index(drop=True)

### Running the algorithm

In [208]:
n = 2

for pi in range(5,p+1):
    run_samples = []

    MAE_dic = {f.__name__:[] for f in functions}
    MAE_l = []
    
    MAE_dic_i = {f.__name__:pd.DataFrame(columns=[r for r in range(run)]) for f in functions}
    
    for r in range (run):
        run_samples.append(df5.iloc[r*(2**pi+5*2**(pi-4)):(r+1)*(2**pi+5*2**(pi-4))].reset_index(drop=True))
        run_samples[-1] = run_samples[-1].T.iloc[df3.iloc[r]].reset_index(drop=True).T

        sample_Matrices = [run_samples[-1].iloc[:,m*int(len(run_samples[-1].T)/n):(m+1)*int(len(run_samples[-1].T)/n)] 
                           for m in range(n)]

        mixed_Matrices = []
        for j in range(k):
            mixed_Matrices.append(sample_Matrices[0].copy())
            mixed_Matrices[-1][j]=sample_Matrices[-1][k+j]

        for f in functions:
            MAE_l = []
            Est_cm1 = pd.DataFrame(index=[j for j in reversed(range(k))])

            E_dic = {j:[] for j in range(k)}

            E_list = []

            c1 = [0]

            for qa in range(2):

                if qa == 0:
                    c1.append(2**(pi-2))

                    qample = sample_Matrices[0].iloc[sum(c1[:-1]):sum(c1)]

                    for j in range(k):
                        E_dic[j] = (0.5*(f(qample)-
                                         f(mixed_Matrices[j].iloc[qample.index]))**2).values

                    E_df = pd.DataFrame.from_dict(E_dic,orient='index').T

                    E_list.append(E_df)

                    Est_cm = np.std(E_df)

                    Est_cm2 = Est_cm.sort_values(ascending=False).iloc[:-2]
                else:
                    tu = [(np.random.choice(sample_Matrices[0].iloc[sum(c1):].index,1,replace=False)[0],np.random.choice(Est_cm2.index,1)[0])
               for it in range(2**(pi-4))]
                    
                    E_dic2 = {ei:[] for ei in Est_cm2.index}
                    for ei in Est_cm2.index:
                        E_dic2[ei]= 0.5*(f(sample_Matrices[0].iloc[sum(c1):])-f(mixed_Matrices[ei].iloc[sum(c1):]))**2
                    
                    E_df2 = pd.concat([E_df,pd.DataFrame.from_dict(E_dic2,orient='index').T]).sort_index()
                    
                    for t in tu:
                        E_df2.at[t]=np.nan

            me2 = E_df2.expanding(1).mean().reset_index(drop=True)

            Var = f(sample_Matrices[0]).expanding(1).var(ddof=0)

            T = (me2.T/Var).T

            AE_v = abs(T-AE_df.loc[f.__name__])
            MAE_r = AE_v.mean(axis=1)
            
            MAE_r.index=(E_df2.count(axis=1)+1).cumsum()
            
            MAE_dic[f.__name__].append(MAE_r)
            
            MAE_dic_i[f.__name__] = pd.concat(MAE_dic[f.__name__],axis=1).interpolate()

    writer = pd.ExcelWriter(str(2**pi)+' New_Estimator_A_config.xlsx', engine='xlsxwriter')

    for mk in MAE_dic_i.keys():
        MAE_dic_i[mk].mean(axis=1).to_excel(writer, sheet_name=mk)

    writer.save()